In [14]:
from classiq import *

In [15]:
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

In [16]:
size = 4 
num_nodes = 2**size 

In [27]:
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)


def C_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    
    num_nodes = 2**4
    prob = [0]*num_nodes  
    if i == 0:
        prob[i + 1] = 1.0 
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0 
    else:
        prob[i - 1] = 0.5  
        prob[i + 1] = 0.5  
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

@qfunc 
def C_operator(vertices:QNum, adjacent_vertices: QNum):
    
    num_nodes = 2**4
    for i in range(num_nodes):
        C_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices - adjacent_vertices) == 1) | ((vertices - adjacent_vertices) == -1))


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))
  

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    size = 4
    allocate(size, vertices)  # Allocate qubits for vertices
    hadamard_transform(vertices)  # Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # Allocate qubits for adjacent vertices

    C_operator(vertices, adjacent_vertices)  # Apply coin operator
    S_operator(vertices, adjacent_vertices)  # Apply shift operator



In [28]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Node=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
Node=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
Node=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
Node=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
Node=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
Node=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
Node=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
Node=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]
Node

The details of the synthesized circuit is below:

Depth: 6154 <br>
Width: 53 <br>
Gate count <br>
H : 1058<br>
X : 657<br>
CU1 : 412<br>
U2 : 256<br>
U1 : 512<br>
CX : 2488<br>
C_OPERATOR_ZERO_DIFFUZER_0_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
RY : 212<br>
TDG : 237<br>
T : 316<br>
P : 1340<br>
C_OPERATOR_ZERO_DIFFUZER_1_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_2_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_3_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_4_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_5_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_6_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_7_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_8_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_9_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_10_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_11_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_12_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_13_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_14_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
C_OPERATOR_ZERO_DIFFUZER_15_ZERO_DIFFUZER_AUX_CONTROL_NAME : 1<br>
CP : 216


The circuit can be optimised and the resulting number of qubits is 35 for the optimized case, but a much greater depth. 